<a href="https://colab.research.google.com/github/alessandronascimento/pyLiBELa/blob/main/Colabs/Validation/SB2021/Compute_Grids.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Downloading dependencies {display-mode: "form"}

%%capture
! apt-get install python-dev-is-python3 zlib1g-dev libeigen3-dev libgsl-dev libnlopt-cxx-dev
!pip install ipython-autotime
%load_ext autotime
!pip3 install condacolab
import condacolab
condacolab.install()
!mamba install openbabel

time: 3.12 s (started: 2023-10-23 20:32:34 +00:00)


In [2]:
#@title Installing pyLiBELa {display-mode: "form"}

%%capture

! rm -f Makefile*
! rm -rf obj src test
! rmdir obj src
! git clone --branch alex-works https://github.com/alessandronascimento/pyLiBELa.git
! wget https://raw.githubusercontent.com/alessandronascimento/pyLiBELa/alex-works/Colabs/Makefile
! mv pyLiBELa/src src
! mv pyLiBELa/test test
! rm -rf pyLiBELa
! mkdir -p obj
! sed -i 's+-I/usr/include/openbabel3+-I/usr/local/include/openbabel3+g' Makefile
! make -j2

time: 2min 13s (started: 2023-10-23 20:32:37 +00:00)


In [1]:
#@title Importing pyLiBELa {display-mode: "form"}

import random
from google.colab import drive

try:
  from pyPARSER import *
  from pyMol2 import *
  from pyWRITER import *
  from pyGrid import *
  from pyCOORD_MC import *
  from pyFindHB import *
  from pyEnergy2 import *
  from pyGaussian import *
  from pyConformer import *
  from pyRAND import *
  from pyMcEntropy import *
  from pySA import *
  from pyOptimizer import *
  from pyMC import *
  from pyFullSearch import *
  from pyDocker import *
  print('pyLiBELa is imported!')
except ImportError:
  print('An ImportError occurred, try running this cell again!')

pyLiBELa is imported!


In [2]:
#@title Getting SB2021 data from Google Drive {display-mode: "form"}

%%capture
drive.mount('/content/drive/')
sb_folder = '/content/drive/MyDrive/Projects_Data/pyLiBELa/SB/' #@param {type:"string"}

%cd $sb_folder
targets=[]
targets_file = open('list', 'r');
for line in targets_file:
  targets.append(line.strip())
targets_file.close()
targets = targets[:-2] #os últimos 2 são list e list.txt, só quero os pdb


In [3]:
#@title Set Parameters for Grid Computation {display-mode: "form"}

Input = PARSER()

Input.generate_conformers = True;
Input.dock_parallel = False;
Input.parallel_jobs = 1;
Input.write_grids = True;
Input.load_grid_from_file = True;
Input.use_grids = True
Input.write_mol2 = True
Input.atom_limit = 10000 #@param {type:"number"}

scoring_function = "0" #@param ["0", "1", "2", "3"]
Input.dielectric_model = "r" #@param ["r", "constant"]
Input.scoring_function = int(scoring_function)
grid_dimension = 30.0 #@param {type:"number"}
Input.grid_prefix = 'McGrid_SF0'
Input.grid_spacing = 0.4 #@param {type:"number"}
Input.solvation_alpha = 0.1 #@param {type:"number"}
Input.solvation_beta = -0.005 #@param {type:"number"}

# Optimization parameter:
Input.min_tol = 1E-4;
Input.min_delta = 1E-4;
Input.dock_min_tol = 1E-4;
search_box = 8.0 #@param {type:"number"}
Input.timeout = 20 #@param {type:"number"}
Input.min_timeout = 30 #@param {type:"number"}
Input.overlay_optimizer = "mma" #@param ["mma", "ln_auglag", "subplex", "none"]
Input.energy_optimizer = "mma" #@param ["direct", "isres", "crs", "esch", "stogo", "mma", "simplex", "none"]
if (Input.scoring_function < 3):
    delta = 2.5 #@param {type:"number"}
    Input.deltaij6 = (delta*delta*delta*delta*delta*delta)
    delta_es = 2.5 #@param {type:"number"}
    Input.deltaij_es6 = pow(delta_es, 6);
    Input.deltaij_es3 = (delta_es*delta_es*delta_es)

Input.conf_search_trials = 10000;
Input.conformers_to_evaluate = 2;
Input.lig_conformers = 10;

Input.search_box_x, Input.search_box_y, Input.search_box_z = search_box, search_box, search_box;
Input.x_dim, Input.y_dim, Input.z_dim = grid_dimension, grid_dimension, grid_dimension;

In [ ]:
#@title Grid Computation {display-mode: "form"}
for target in targets:
  target_folder = sb_folder + target
  %cd $target_folder
  !mkdir -p grids
  %cd grids
  try:
    REC = Mol2(Input, '../rec.mol2.gz')
    RefLig = Mol2(Input, '../lig.mol2.gz')
    Lig2 = Mol2(Input, 'lig.mol2.gz')
    Writer = WRITER(Input)
    Coord = COORD_MC()
    HB = FindHB()
    Ene  = Energy2(Input)
    for i in range(len(REC.residue_pointer)-1):
      HB.parse_residue(REC.residue_pointer[i]-1, REC.residue_pointer[i+1]-2, REC.resnames[i], REC, RefLig, 9.0);
    HB.find_ligandHB('../lig.mol2.gz', RefLig);
    HB.find_ligandHB('../lig.mol2.gz', Lig2);
    print('The receptor has %5d / %5d HB donors/acceptors around the active site.' % (len(REC.HBdonors), len(REC.HBacceptors)));
    center = Coord.compute_com(RefLig)

    print()
    start_energy = Ene.compute_ene(REC, RefLig, RefLig.xyz);
    print('Starting energy: %7.3f kcal/mol' % start_energy);
    print()
    print('Generating grids. This may take a while..')
    Grids = Grid(Input, Writer, REC, center)
    print('Grids computed!')
    grid_energy = Ene.compute_ene(Grids, RefLig, RefLig.xyz);
    print('Grid original energy: %7.3f kcal/mol' % grid_energy);
    print('Grid error: %7.3f %%' % abs(100.*(start_energy-grid_energy)/start_energy));

  except:
    print('Could not compute grids for target %s.' % target)






/content/drive/.shortcut-targets-by-id/1iX13EV9pFTj4bafAE1ugNFli9exvXFbC/pyLiBELa/SB/121P
/content/drive/.shortcut-targets-by-id/1iX13EV9pFTj4bafAE1ugNFli9exvXFbC/pyLiBELa/SB/121P/grids
The receptor has    24 /    25 HB donors/acceptors around the active site.

Starting energy: -144.121 kcal/mol

Generating grids. This may take a while..
Grids computed!
Grid original energy: -133.592 kcal/mol
Grid error:   7.306 %
/content/drive/.shortcut-targets-by-id/1iX13EV9pFTj4bafAE1ugNFli9exvXFbC/pyLiBELa/SB/181L
/content/drive/.shortcut-targets-by-id/1iX13EV9pFTj4bafAE1ugNFli9exvXFbC/pyLiBELa/SB/181L/grids
The receptor has    38 /    26 HB donors/acceptors around the active site.

Starting energy: -12.525 kcal/mol

Generating grids. This may take a while..
Grids computed!
Grid original energy: -12.457 kcal/mol
Grid error:   0.540 %
/content/drive/.shortcut-targets-by-id/1iX13EV9pFTj4bafAE1ugNFli9exvXFbC/pyLiBELa/SB/182L
/content/drive/.shortcut-targets-by-id/1iX13EV9pFTj4bafAE1ugNFli9exvXFbC/pyL